In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Taking the address of the each tim hortons 
def address_Addition(address1,new_df):
    address_list = address1.split(',')
    new_df['address'] = address_list[0]
    new_df['city'] = address_list[1]
    return new_df

driver = webdriver.Chrome()
url = ['https://www.google.com/maps/place/Tim+Hortons/@43.7607366,-79.5321831,14z/data=!4m10!1m2!2m1!1stim+hortons!3m6!1s0x882b31d93eab2809:0xa9ea7bb65f9da6ec!8m2!3d43.7607366!4d-79.4992241!15sCgt0aW0gaG9ydG9ucyIDiAEBWg0iC3RpbSBob3J0b25zkgEKcmVzdGF1cmFudOABAA!16s%2Fg%2F1vyxk0xz','https://www.google.com/maps/place/Tim+Hortons/@43.7607366,-79.5321831,14z/data=!4m10!1m2!2m1!1stim+hortons!3m6!1s0x882b302d70a29891:0xc279061e4a5c71bc!8m2!3d43.756124!4d-79.5152637!15sCgt0aW0gaG9ydG9ucyIDiAEBWg0iC3RpbSBob3J0b25zkgEKcmVzdGF1cmFudOABAA!16s%2Fg%2F1td38wkb']
c = 0
for i in range(0,len(url)):
    c = c+1
    driver.get(url[i])
    time.sleep(5)

    # Finding the address of the location
    response = BeautifulSoup(driver.page_source, 'html.parser')
    address = response.find('div',class_= 'rogA2c').text
    driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]').click()
    time.sleep(3)
    SCROLL_PAUSE_TIME = 5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    number = 0

    while True:
        number = number+1

        # Scroll down to bottom

        #old_==ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
        ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]')
        driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)

        # Wait to load page

        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        #print(f'last height: {last_height}')

        ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]')

        new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

        #print(f'new height: {new_height}')

        if number == 5:
            break

        if new_height == last_height:
            break

        #print('cont')
        last_height = new_height
    next_item = driver.find_elements('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]')
    time.sleep(3)

    for i in next_item:
        button = i.find_elements(By.TAG_NAME,'button')
        for m in button:
            if m.text == "More":
                m.click()
        time.sleep(5)

    response = BeautifulSoup(driver.page_source, 'html.parser')
    next_2 = response.find_all('div',class_ = 'jftiEf')


    def get_review_summary(result_set):
        rev_dict = {'Review Name': [],
            'Review Text' : []}

        for result in result_set:
            review_name = result.find(class_='d4r55').text
            review_text = result.find('span',class_='wiI7pd').text
            rev_dict['Review Name'].append(review_name)
            rev_dict['Review Text'].append(review_text)
         
        return(pd.DataFrame(rev_dict))

    df = get_review_summary(next_2)
    if c == 1:
        df1 = df.copy()
        final_df = address_Addition(address,df1)
    else:
        df2 = df.copy()
        final_df = address_Addition(address,df2)
        final_df1 = pd.concat([df1,final_df],axis = 0)
    

print(final_df1)

           Review Name                                        Review Text  \
0       Shweta Jindal   This is how I got the order from this place. I...   
1         Nero Odogun   Got my order wrong which happens. Had a sausag...   
2      Eat with Arpit   They don't even have coffee there. I went ther...   
3       Akhilesh Dave   I have ordered Cheese croissant toasted. The l...   
4         Cara Cooper   In TO for a couple of weeks for a course. This...   
..                 ...                                                ...   
55       Camilla Leon   Hello it great to go get some early coffee tas...   
56    Jackie Baechler   This Tim Horton's is amazing, service is quick...   
57   maxwell mitchell   Good place to have a cup of Tim's and hang out...   
58         Don Joseph                      Busy place, always good coffee   
59       Ramin Yamini   Good place to get a coffee or light food with ...   

              address         city  
0    1493 Finch Ave W   North York  
1

In [2]:
final_df1

,Review Name,Review Text,address,city
0,Shweta Jindal,This is how I got the order from this place. I...,1493 Finch Ave W,North York
1,Nero Odogun,Got my order wrong which happens. Had a sausag...,1493 Finch Ave W,North York
2,Eat with Arpit,They don't even have coffee there. I went ther...,1493 Finch Ave W,North York
3,Akhilesh Dave,I have ordered Cheese croissant toasted. The l...,1493 Finch Ave W,North York
4,Cara Cooper,In TO for a couple of weeks for a course. This...,1493 Finch Ave W,North York
...,...,...,...,...
55,Camilla Leon,Hello it great to go get some early coffee tas...,3981 Jane St,North York
56,Jackie Baechler,"This Tim Horton's is amazing, service is quick...",3981 Jane St,North York
57,maxwell mitchell,Good place to have a cup of Tim's and hang out...,3981 Jane St,North York
58,Don Joseph,"Busy place, always good coffee",3981 Jane St,North York


In [3]:
final_df1.to_csv('web_scraping.csv')
